# affine invariant ensemble sampler + AIS

### 12D transporter model (single antiporter cycle)

Annealed Importance Sampling w/ the Affine Invariant Sampler

1. Initialization
    1. set $\mathrm{ESS_{min}}$, $\beta=0$, $\Delta\beta$, $N$ steps, $K$ walkers, last fraction of samples to keep $\alpha$ 
        1. note: choose small $\Delta\beta$ 
        2. note: choose  $(\alpha \cdot N\cdot K)$>>$K$
        3. note: need to let samples become 'decorrelated' $\rightarrow$ select some fraction of samples at the end
    2. generate $\alpha \cdot N\cdot K$ initial samples from a Uniform distribution (same as priors)
    3. calculate likelihoods at initial sample points ($\beta$=0), $\mathcal{L_{0}}$
2. Determine new $\beta$ 
    1. WHILE $\mathrm{ESS}$ > $\mathrm{ESS_{min}}$:
        1. increment $\beta$:  $\beta_{\mathrm{new}} \rightarrow \beta + \Delta\beta$
        2. calculate $\mathrm{ESS}$:
            1. calculate weights $w$: $w(x) = \mathcal{L}^{(\beta_{\mathrm{new}}-\beta)}$
                1. note: using log likelihood, so use relative weights:
                    1. $\ln(w) = (\beta_{\mathrm{new}}-\beta) \ln(\mathcal{L_{\mathrm{ref}}})$
                    2. $\ln(w)_{\mathrm{rel}} = \ln(w)_{\mathrm{i}} - \max{\ln(w)}$ 
            2. calculate $\mathrm{ESS}$:
                1. $\mathrm{ESS} = \sum{\frac{w_i}{\max{w}}} = \sum(\exp(\ln(w)_{\mathrm{rel}}))$
3. Run sampler
    1. resample $K$ walkers, drawn with replacement from initial parameter set distribution using weights, $w$.
    2. run (affine invarient ensemble) sampler for $N\cdot K$ total samples
    3. select the last fraction of samples to keep $\alpha \cdot N\cdot K$ 
    4. calculate likelihood for the above samples
4. repeat 2-3 until $\beta=1$
    1. (possibly run sampler longer when $\beta=1$)


In [ ]:
import numpy as np, tellurium as te, matplotlib.pyplot as plt
import emcee as mc, corner, time, pandas as pd
np.random.seed(10)


# Normal log-likelihood calculation
def calc_norm_log_likelihood(mu,sigma,X):
    # Normal log-likelihood function: -[(n/2)ln(2pp*sigma^2)]-[sum((X-mu)^2)/(2*sigma^2)]
    # ref: https://www.statlect.com/fundamentals-of-statistics/normal-distribution-maximum-likelihood 
    n = len(X)
    f1 = -1*(n/2)*np.log(2*np.pi*sigma**2)
    f2_a = -1/(2*sigma**2)
    f2_b = 0 
    for i in range(n):
        f2_b += (X[i]-mu[i])**2
    f2 = f2_a*f2_b
    log_likelihood = f1+f2
    return log_likelihood


def calc_grid_point(K,y_obs,m):
    m.resetToOrigin()
    m.H_out_activation = 5e-8
    m.integrator.absolute_tolerance = 1e-18
    m.integrator.relative_tolerance = 1e-12
    m.k1_f = 10**K[0]
    m.k1_r = 10**K[1]
    m.k6_r = (m.k1_f*m.k2_f*m.k3_f*m.k4_f*m.k5_f*m.k6_f)/(m.k1_r*m.k2_r*m.k3_r*m.k4_r*m.k5_r)
 
    D_tmp = m.simulate(0,15, 376, selections=['time', 'rxn4'])
    y_tmp = D_tmp['rxn4']
    sigma = 1e-13
    log_like_tmp = calc_norm_log_likelihood(y_tmp,sigma,y_obs)
    return log_like_tmp


antimony_string = f"""
            // Created by libAntimony v2.12.0
            model transporter_full()

            // Compartments and Species:
            compartment vol;
            species OF in vol, OF_Hb in vol;
            species IF_Hb in vol, IF_Hb_Sb in vol;
            species IF_Sb in vol, OF_Sb in vol;
            species H_in in vol, S_in in vol;
            species $H_out in vol, $S_out in vol;

            // Reactions:
            rxn1: OF + $H_out -> OF_Hb; vol*(k1_f*OF*H_out - k1_r*OF_Hb);
            rxn2: OF_Hb -> IF_Hb; vol*(k2_f*OF_Hb - k2_r*IF_Hb);
            rxn3: IF_Hb + S_in -> IF_Hb_Sb; vol*(k3_f*IF_Hb*S_in - k3_r*IF_Hb_Sb);
            rxn4: IF_Hb_Sb -> IF_Sb + H_in; vol*(k4_f*IF_Hb_Sb - k4_r*IF_Sb*H_in);
            rxn5: IF_Sb -> OF_Sb; vol*(k5_f*IF_Sb - k5_r*OF_Sb);
            rxn6: OF_Sb -> OF + $S_out; vol*(k6_f*OF_Sb - k6_r*OF*S_out);
            

            // Events:
            E1: at (time >= 5): H_out = H_out_activation, S_out = S_out_activation;
            E2: at (time >= 10): H_out = 1e-7, S_out = 0.001;

            // Species initializations:
            H_out = 1e-07;
            H_out has substance_per_volume;

            H_in = 1e-7;
            H_in has substance_per_volume;

            S_out = 0.001;
            S_out has substance_per_volume;

            S_in = 1e-3;
            S_in has substance_per_volume;

            OF = 2.833e-8;
            OF has substance_per_volume;

            OF_Hb = 2.833e-8;
            OF_Hb has substance_per_volume;

            IF_Hb = 2.833e-8;
            IF_Hb has substance_per_volume;
            
            IF_Hb_Sb = 2.833e-8;
            IF_Hb_Sb has substance_per_volume;
            
            IF_Sb = 2.125e-08;
            IF_Sb has substance_per_volume;

            OF_Sb = 2.125e-08;
            OF_Sb has substance_per_volume;


            // Compartment initializations:
            vol = 0.0001;
            vol has volume;

            // Variable initializations:
            H_out_activation = 5e-8;
            S_out_activation = 0.001;

            // Rate constant initializations:
            k1_f = 1e10;
            k1_r = 1e3;
            k2_f = 1e2;
            k2_r = 1e2;
            k3_f = 1e7;
            k3_r = 1e3;
            k4_f = 1e3;
            k4_r = 1e10;
            k5_f = 1e2;
            k5_r = 1e2;
            k6_f = 1e3;
            k6_r = 1e7;


            // Other declarations:
            const vol;
            const k1_f, k1_r, k2_f, k2_r, k3_f, k3_r;
            const k4_f, k4_r, k5_f, k5_r, k6_f, k6_r;
    

            // Unit definitions:
            unit substance_per_volume = mole / litre;
            unit volume = litre;
            unit length = metre;
            unit area = metre^2;
            unit time_unit = second;
            unit substance = mole;
            unit extent = mole;

            // Display Names:
            time_unit is "time";
            end
            """ 

antimony_string3 = f"""
            // Created by libAntimony v2.12.0
            model transporter_full()

            // Compartments and Species:
            compartment vol;
            species OF in vol, OF_Hb in vol;
            species IF_Hb in vol, IF_Hb_Sb in vol;
            species IF_Sb in vol, OF_Sb in vol;
            species H_in in vol, S_in in vol;
            species $H_out in vol, $S_out in vol;

            // Reactions:
            rxn1: OF + $H_out -> OF_Hb; vol*(k1_f*OF*H_out - k1_r*OF_Hb);
            rxn2: OF_Hb -> IF_Hb; vol*(k2_f*OF_Hb - k2_r*IF_Hb);
            rxn3: IF_Hb + S_in -> IF_Hb_Sb; vol*(k3_f*IF_Hb*S_in - k3_r*IF_Hb_Sb);
            rxn4: IF_Hb_Sb -> IF_Sb + H_in; vol*(k4_f*IF_Hb_Sb - k4_r*IF_Sb*H_in);
            rxn5: IF_Sb -> OF_Sb; vol*(k5_f*IF_Sb - k5_r*OF_Sb);
            rxn6: OF_Sb -> OF + $S_out; vol*(k6_f*OF_Sb - k6_r*OF*S_out);
            

            // Events:
            E1: at (time >= 5): H_out = 1e-7, S_out = 1e-3;
            

            // Species initializations:
            H_out = 5e-8;
            H_out has substance_per_volume;

            H_in = 9.999811082242941e-08;
            H_in has substance_per_volume;

            S_out = 0.001;
            S_out has substance_per_volume;

            S_in = 0.0009999811143288836;
            S_in has substance_per_volume;

            OF = 4.7218452046117796e-09;
            OF has substance_per_volume;

            OF_Hb = 4.7218452046117796e-09;
            OF_Hb has substance_per_volume;

            IF_Hb = 4.7218452046117796e-09;
            IF_Hb has substance_per_volume;
            
            IF_Hb_Sb = 4.721756029392908e-08;
            IF_Hb_Sb has substance_per_volume;
            
            IF_Sb = 4.721845204611779e-08;
            IF_Sb has substance_per_volume;

            OF_Sb = 4.721845204611775e-08;
            OF_Sb has substance_per_volume;


            // Compartment initializations:
            vol = 0.0001;
            vol has volume;

            // Rate constant initializations:
            k1_f = 1e10;
            k1_r = 1e3;
            k2_f = 1e2;
            k2_r = 1e2;
            k3_f = 1e7;
            k3_r = 1e3;
            k4_f = 1e3;
            k4_r = 1e10;
            k5_f = 1e2;
            k5_r = 1e2;
            k6_f = 1e3;
            k6_r = 1e7;


            // Other declarations:
            const vol;
            const k1_f, k1_r, k2_f, k2_r, k3_f, k3_r;
            const k4_f, k4_r, k5_f, k5_r, k6_f, k6_r;
    

            // Unit definitions:
            unit substance_per_volume = mole / litre;
            unit volume = litre;
            unit length = metre;
            unit area = metre^2;
            unit time_unit = second;
            unit substance = mole;
            unit extent = mole;

            // Display Names:
            time_unit is "time";
            end
            """ 


antimony_string2 = f"""
            // Created by libAntimony v2.12.0
            model transporter_full()

            // Compartments and Species:
            compartment vol;
            species OF in vol, OF_Hb in vol;
            species IF_Hb in vol, IF_Hb_Sb in vol;
            species IF_Sb in vol, OF_Sb in vol;
            species H_in in vol, S_in in vol;
            species $H_out in vol, $S_out in vol;

            // Reactions:
            rxn1: OF + $H_out -> OF_Hb; vol*(k1_f*OF*H_out - k1_r*OF_Hb);
            rxn2: OF_Hb -> IF_Hb; vol*(k2_f*OF_Hb - k2_r*IF_Hb);
            rxn3: IF_Hb + S_in -> IF_Hb_Sb; vol*(k3_f*IF_Hb*S_in - k3_r*IF_Hb_Sb);
            rxn4: IF_Hb_Sb -> IF_Sb + H_in; vol*(k4_f*IF_Hb_Sb - k4_r*IF_Sb*H_in);
            rxn5: IF_Sb -> OF_Sb; vol*(k5_f*IF_Sb - k5_r*OF_Sb);
            rxn6: OF_Sb -> OF + $S_out; vol*(k6_f*OF_Sb - k6_r*OF*S_out);
            

            // Species initializations:
            H_out = 1e-07;
            H_out has substance_per_volume;

            H_in = 1e-7;
            H_in has substance_per_volume;

            S_out = 0.001;
            S_out has substance_per_volume;

            S_in = 1e-3;
            S_in has substance_per_volume;

            OF = 2.833e-8;
            OF has substance_per_volume;

            OF_Hb = 2.833e-8;
            OF_Hb has substance_per_volume;

            IF_Hb = 2.833e-8;
            IF_Hb has substance_per_volume;
            
            IF_Hb_Sb = 2.833e-8;
            IF_Hb_Sb has substance_per_volume;
            
            IF_Sb = 2.125e-08;
            IF_Sb has substance_per_volume;

            OF_Sb = 2.125e-08;
            OF_Sb has substance_per_volume;


            // Compartment initializations:
            vol = 0.0001;
            vol has volume;


            // Rate constant initializations:
            k1_f = 1e10;
            k1_r = 1e3;
            k2_f = 1e2;
            k2_r = 1e2;
            k3_f = 1e7;
            k3_r = 1e3;
            k4_f = 1e3;
            k4_r = 1e10;
            k5_f = 1e2;
            k5_r = 1e2;
            k6_f = 1e3;
            k6_r = 1e7;


            // Other declarations:
            const vol;
            const k1_f, k1_r, k2_f, k2_r, k3_f, k3_r;
            const k4_f, k4_r, k5_f, k5_r, k6_f, k6_r;
    

            // Unit definitions:
            unit substance_per_volume = mole / litre;
            unit volume = litre;
            unit length = metre;
            unit area = metre^2;
            unit time_unit = second;
            unit substance = mole;
            unit extent = mole;

            // Display Names:
            time_unit is "time";
            end
            """ 



m = te.loada(antimony_string)
m.integrator.absolute_tolerance = 1e-18
m.integrator.relative_tolerance = 1e-12



D = m.simulate(0, 15, 376, selections=['time', 'rxn4'])
#y_true = D['rxn4']

m.resetToOrigin()
m.H_out_activation = 2e-7
m.integrator.absolute_tolerance = 1e-18
m.integrator.relative_tolerance = 1e-12
D2 = m.simulate(0, 15, 376, selections=['time', 'rxn4'])



y_true = np.hstack([D['rxn4'],D2['rxn4']])

noise_stdev_true = 1e-13
y_obs = y_true + np.random.normal(0, noise_stdev_true, len(y_true))
#y_obs = np.genfromtxt("data_grid_test3_2.csv", skip_header=1)

plt.figure(figsize=(10,10))
plt.plot(y_obs, 'o', alpha=0.25)
plt.plot(y_true)
plt.ylim(-2.5e-11, 2.5e-11)

log_like_ref = calc_norm_log_likelihood(y_true,noise_stdev_true,y_obs)

print(log_like_ref)

m2 = te.loada(antimony_string3)
m2.integrator.absolute_tolerance = 1e-18
m2.integrator.relative_tolerance = 1e-12
m2.H_out = 5e-8
D3 = m2.simulate(0, 10, 250, selections=['time', 'rxn4'])
m2.resetToOrigin()
m2.H_out = 2e-7
m2.integrator.absolute_tolerance = 1e-18
m2.integrator.relative_tolerance = 1e-12
D4 = m2.simulate(0, 10, 250, selections=['time', 'rxn4'])
y_true2 = np.hstack([D3['rxn4'],D4['rxn4']])
y_obs2 = y_true2 + np.random.normal(0, noise_stdev_true, len(y_true2))
plt.figure(figsize=(10,10))
plt.plot(y_obs2, 'o', alpha=0.25)
plt.plot(y_true2)
plt.ylim(-2.5e-11, 2.5e-11)
np.savetxt("data_grid_test3_2exp.csv", y_obs2, delimiter=",")

log_like_ref = calc_norm_log_likelihood(y_true2,noise_stdev_true,y_obs2)
print(log_like_ref)

In [ ]:
# Algorithm outline: 

### Functions

# Normal log-likelihood calculation
def calc_norm_log_likelihood(mu,sigma,X):
    # Normal log-likelihood function: -[(n/2)ln(2pp*sigma^2)]-[sum((X-mu)^2)/(2*sigma^2)]
    # ref: https://www.statlect.com/fundamentals-of-statistics/normal-distribution-maximum-likelihood 
    n = len(X)
    f1 = -1*(n/2)*np.log(2*np.pi*sigma**2)
    f2_a = -1/(2*sigma**2)
    f2_b = 0 
    for i in range(n):
        f2_b += (X[i]-mu[i])**2
    f2 = f2_a*f2_b
    log_likelihood = f1+f2
    return log_likelihood


# simulate model and then compare flux data to observed data
def log_likelihood(theta, y_obs, extra_parameters):
    '''log of Guassian likelihood distribution'''
    sigma = 10**theta[0]
    K=theta[1:]
    m = extra_parameters[0] 
    m.resetToOrigin()
    m.H_out_activation = 5e-8
    m.integrator.absolute_tolerance = 1e-18
    m.integrator.relative_tolerance = 1e-12
    m.k1_f = 10**K[0]
    m.k1_r = 10**K[1]
    m.k2_f = 10**K[2]
    m.k2_r = 10**K[3]
    m.k3_f = 10**K[4]
    m.k3_r = 10**K[5]
    m.k4_f = 10**K[6]
    m.k4_r = 10**K[7]
    m.k5_f = 10**K[8]
    m.k5_r = 10**K[9]
    m.k6_f = 10**K[10]
    m.k6_r = (m.k1_f*m.k2_f*m.k3_f*m.k4_f*m.k5_f*m.k6_f)/(m.k1_r*m.k2_r*m.k3_r*m.k4_r*m.k5_r)
    try:
        D_tmp = m.simulate(0,15, 376, selections=['time', 'rxn4'])
        y_tmp = D_tmp['rxn4']
        log_like_tmp = calc_norm_log_likelihood(y_tmp,sigma,y_obs)
    except:
        log_like_tmp = -np.inf
    return log_like_tmp


# calculate uniform prior
def log_prior(theta):
    '''log of uniform prior distribution'''
    p0 = theta[0]
    p1 = theta[1]
    p2 = theta[2]
    p3 = theta[3]
    p4 = theta[4]
    p5 = theta[5]
    p6 = theta[6]
    p7 = theta[7]
    p8 = theta[8]
    p9 = theta[9]
    p10 = theta[10]
    p11 = theta[11]

    # if prior is between boundary --> log(prior) = 0 (uninformitive prior)
    if (np.log10(5e-14)<p0<np.log10(5e-13)) and (6<p1<12) and (-1<p2<5) and (-2<p3<4) and (-2<p4<4) and \
        (3<p5<9) and (-1<p6<5) and (-1<p7<5) and (6<p8<12) and (-2<p9<4) and (-2<p10<4) and (-1<p11<5):
        return 0  
    else:
        return -np.inf

    
# calculate log probability (log likelihood + log prior)
def log_probability(theta, y_obs, extra_parameters):
    '''log of estimated posterior probability'''
    B = extra_parameters[1]  # beta term
    log_pr = log_prior(theta)
    if not np.isfinite(log_pr):
        return -np.inf  # ~zero probability
    log_like = log_likelihood(theta, y_obs, extra_parameters)**B
    log_post = log_pr + log_like
    return log_post


# set initial parameters
def set_p0():
    log_noise_sigma = np.random.uniform(np.log10(5e-14), np.log10(5e-13))
    log_k1_f = np.random.uniform(6, 12) # log10 rate constant (ref=1e10)
    log_k1_r = np.random.uniform(-1,5)  # log10 rate constant (ref=1e3) 
    log_k2_f = np.random.uniform(-2,4)  # log10 rate constant (ref=1e2)
    log_k2_r = np.random.uniform(-2,4)  # log10 rate constant (ref=1e2)

    log_k3_f = np.random.uniform(3,9)  # log10 rate constant (ref=1e7) 
    log_k3_r = np.random.uniform(-1,5)  # log10 rate constant (ref=1e3) 
    log_k4_f = np.random.uniform(-1,5)  # log10 rate constant (ref=1e3) 
    log_k4_r = np.random.uniform(6, 12)  # log10 rate constant (ref=1e10)
    log_k5_f = np.random.uniform(-2,4)  # log10 rate constant (ref=1e2)
    log_k5_r = np.random.uniform(-2,4)   # log10 rate constant (ref=1e2)
    log_k6_f = np.random.uniform(-1,5)  # log10  rate constant (ref=1e3)
    
    p0_list_tmp = [
                log_noise_sigma ,
                log_k1_f ,
                log_k1_r ,
                log_k2_f ,
                log_k2_r ,
                log_k3_f , 
                log_k3_r ,
                log_k4_f ,
                log_k4_r ,
                log_k5_f ,
                log_k5_r ,
                log_k6_f ,
    ]
    return p0_list_tmp


# calculate a new beta 
def calc_beta(beta_old, d_beta, log_like_ref):
    
    beta_new = beta_old + d_beta
    # figure out converting from log likelihood to weights
    
    
    w_new = (np.sign(log_like_ref))*(np.abs(log_like_ref)**(beta_new-beta_old))
    
    print(beta_old)
    print(d_beta)
    print(beta_new)
    print(w_new[:2])
    pass


def calc_weights(log_like_ref, beta1, beta2):
    log_w = (beta2-beta1)*log_like_ref  # natural log
    print(log_w)
    

### info for parameters: name, log10 lower bound, log10 upper bound, reference value
p_info = [
    ["log_sigma",np.log10(5e-14), np.log10(5e-13), -13],
    ["log_k1_f",6,12,10],
    ["log_k1_r",-1,5,3],
    ["log_k2_f",-2,4,2],
    ["log_k2_r",-2,4,2],
    ["log_k3_f",3,9,7],
    ["log_k3_r",-1,5,3],
    ["log_k4_f",-1,5,3],
    ["log_k4_r",6,12,10],
    ["log_k5_f",-2,4,2],
    ["log_k5_r",-2,4,2],
    ["log_k6_f",-1,5,3],
]

### Initialize ESS_min, β=0, Δβ, N samples
seed = 10
np.random.seed(seed)
ESS_min = 100
beta = 1
d_beta = 0.1
N_samples = 120
extra_parameters = [m,beta]

### reference log likelihood
p_ref = [-13,10,3,2,2,7,3,3,10,2,2,3]  # log10 parameter values (sigma, k's)
print(f"ref log likelihood: {log_likelihood(p_ref,y_obs,extra_parameters)}")

### initial sample at beta = 0
# Sample from uniform likelihood: L(D|θ)^0
# Latin hypercube, low-discrepancy sequence ?
N_dim = 12
N_steps = 5000
N_walkers = N_samples

pos_list = []
for i in range(N_walkers):
    p0_list_tmp = set_p0()
    pos_list.append(p0_list_tmp)
start_pos = np.asarray(pos_list)

sampler = mc.EnsembleSampler(N_walkers, N_dim, log_probability,  args=(y_obs, extra_parameters))
t_0 = time.time()
pos, lnprob, rstate = sampler.run_mcmc(start_pos, n_steps)
t_run = time.time() - t_0
lp = sampler.lnprobability
samples = [np.transpose(sampler.flatchain[0,:])]
print(f"beta: {beta}")
print(f"wall clock: {t_run} s")
print(f"{np.shape(lp)[0]*np.shape(lp)[1]/t_run} likelihood calculations / sec " )

df = pd.DataFrame(sampler.flatchain[:,:], columns=[p_info[i][0] for i in range(len(p_info))])
df["lp"] = np.array(lp).flatten()
print(df)
beta=0

calc_beta(beta,d_beta, df.iloc[:,12].values)



# Calculate β_new such that ESS(β_new) > ESS_min
# β_new = β + Δβ
# Calculate weights: w ~ L^(β_new – β). [ratio of Gaussian distributions ]
# Calculate ESS: sum(weights)/max(weights)
# Resample N points using weights from β_new 
# Run sampler (i.e. affine invariant ensemble) from resampled points
# Repeat steps 3 to 5 until β=1




In [ ]:
### 1D posterior plot
fig, ax = plt.subplots(3,4, figsize=(20,15))
axes = ax.flatten()

for i, ax_i in enumerate(axes):
    p_tmp = p_info[i]
    ax_i.set_title(f"{p_tmp[0]}")
    ax_i.axvline(p_tmp[3],ls='--', color='black', alpha=0.5)
    ax_i.set_xlim(p_tmp[1],p_tmp[2])
    

    ax_i.hist(df.iloc[:, i], 100, histtype="step", density=True, range=(p_tmp[1],p_tmp[2]), label=f'{p_tmp[0]}')
    ax_i.legend()
    
plt.tight_layout()
#plt.savefig(f'12D_transporter_AIES_PT_1D_dist_s{seed}_random.png')

In [ ]:
import pandas as pd
df1 = pd.read_csv("affine_2exp_p0_max_nw_500_ns_100000_data.csv", usecols=[i+1 for i in range(N_dim)])
df2 = pd.read_csv("affine_2exp_p0_max_nw_500_ns_100000_s11_data.csv", usecols=[i+1 for i in range(N_dim)])

import corner
p_info = [
    ["log_sigma",np.log10(5e-14), np.log10(5e-13), -13],
    ["log_k1_f",6,12,10],
    ["log_k1_r",-1,5,3],
    ["log_k2_f",-2,4,2],
    ["log_k2_r",-2,4,2],
    ["log_k3_f",3,9,7],
    ["log_k3_r",-1,5,3],
    ["log_k4_f",-1,5,3],
    ["log_k4_r",6,12,10],
    ["log_k5_f",-2,4,2],
    ["log_k5_r",-2,4,2],
    ["log_k6_f",-1,5,3],
]

ranges = [(p_info[i][1],p_info[i][2]) for i in range(len(p_info))]
truths = [p_info[i][3] for i in range(len(p_info))]
labels = [p_info[i][0] for i in range(len(p_info))]

fig = corner.corner(df, bins=200, range=ranges, truths=truths, titles=labels, show_titles=True)
savefig('12d_transporter_AIS_PT_pair_plot.png')

In [ ]:


class AIS_Affine_Sampler:
    def __init__(self, ESS_min, d_beta, N_steps, K_walkers, alpha, beta_schedule, verbose):
        self.ESS_min = float(ESS_min)
        self.d_beta = float(d_beta)
        self.N_steps = int(N_steps)
        self.K_walkers = int(K_walkers)
        self.alpha = int(alpha)
        self.beta_schedule = beta_schedule
        self.verbose = verbose
        
        self.M_samples = int(self.N_steps*self.K_walkers)
        self.S_subsamples = int(self.alpha*self.K_walkers)
        
        # check if using adaptive beta schedule or fixed beta schedule
        if not self.beta_schedule:
            if self.verbose == True:
                print('> beta schedule is empty - using adaptive method')
            self.beta_adapt = True
        else:
            self.beta_adapt =False
        
        # check if hyper parameters are valid
        assert(self.M_samples > self.S_subsamples >= 10*self.K_walkers), ("invalid number of subsamples")
        assert(0.1>self.d_beta<1), ("invalid delta beta")
        
    
        

        
        

ESS_min = 10
beta_schedule = []
d_beta = 0.01
N_steps = 100
K_walkers = 10
alpha = 10
verbose = True

sampler = AIS_Affine_Sampler(ESS_min,d_beta, N_steps, K_walkers, alpha, beta_schedule, verbose)
print(vars(sampler))

In [ ]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import emcee
import matplotlib.pyplot as plt


### bimodal Gaussian in 1D

def prior(x):
    if np.all(x>=-20) and np.all(x<=20):
        return np.ones(np.size(x))
    else:
        return np.zeros(np.size(x))

    
def likelihood(x, mu, sigma):
    L = 0.5*(1/(sigma[0] * np.sqrt(2 * np.pi))*np.exp(-(x-mu[0])**2 / (2 * sigma[0]**2))) + 0.5*(1/(sigma[1] * np.sqrt(2 * np.pi))*np.exp(-(x-mu[1])**2 / (2 * sigma[1]**2)))
    return L


def log_inf(x):
    return np.where(x != 0, np.log(x), -np.inf) 


def log_prob(x, beta, mu, sigma):
    pr_x = prior(x) 
    like_x = likelihood(x, mu, sigma)
    log_pr = log_inf(pr_x)
    log_like = log_inf(like_x)  
    return log_pr + beta*log_like
### 

def OVL(pdf1, pdf2):
    # calculate overlapping coefficient
    assert(len(pdf1)==len(pdf2))
    ovl_coeff = 0
    for i in range(len(pdf1)):
        ovl_coeff += np.min(pdf1[i], pdf2[i])
    return ovl_coeff
        

### analytical model
def prob(x,beta, mu, sigma):
    pr = 1.0
    L = 0.5*(1/(sigma[0] * np.sqrt(2 * np.pi))*np.exp(-(x-mu[0])**2 / (2 * sigma[0]**2))) + 0.5*(1/(sigma[1] * np.sqrt(2 * np.pi))*np.exp(-(x-mu[1])**2 / (2 * sigma[1]**2)))
    
    p = pr*(L**beta)
    bin_width = np.size(x)/np.ptp(x)
    p_sum = np.array([])
    
    return (pr*(L**beta))

### model configuration
mu = [-10,10]
sigma = [1,2]
x = np.linspace(-20,20,100)
beta = [0, 0.1, 1]
#y_beta = [np.exp(log_prob(x, b, mu, sigma)) for b in beta]  # problem here


### sampling settings
np.random.seed(10)
dim = 1
max_iter = 100
ESS_min = 500

beta=0
d_beta = 0.01
N_steps = 10000
K_walkers = 100
M_sub_samples = 10*K_walkers
NK_total_samples = N_steps*K_walkers


### step 1: initialization
S = (np.ptp(x)*np.random.rand(alpha))-np.max(x)  

### main loop
j = 0
while (beta< 1) and (j < max_iter):
    assert(np.size(S)==M_sub_samples)
    i = 0
    S = S.flatten()
    ESS = np.inf
    log_p1 = log_prob(S,beta,mu,sigma)
    
    beta_tmp_list = []
    log_p2_tmp_list = []
    w_rel_tmp_list = []
    ESS_tmp_list = []
    
    beta_new = beta
    while (beta_new <= 1) and (ESS > ESS_min) and (i < max_iter):
        beta_new = beta+d_beta*i
        log_p2 = log_prob(S,beta_new,mu, sigma)
        log_w = log_p2 - log_p1
        log_w_rel = log_w-np.max(log_w)
        w_rel = np.exp(log_w_rel)
        p_rel = w_rel/np.sum(w_rel)
        ESS = np.sum(w_rel)
        print(f'average relative probability: {np.mean(p_rel)}')
        beta_tmp_list.append(beta_new)
        log_p2_tmp_list.append(log_p2)
        w_rel_tmp_list.append(p_rel)
        ESS_tmp_list.append(ESS)
        print(f'beta new = {beta_new}')
        print(f'~ESS (sum of relative probability) = {ESS} samples')
        i += 1


    beta=beta_tmp_list[-2]
    rel_w_list = w_rel_tmp_list[-2]

    re_S = np.random.choice(S,size=K_walkers,p=rel_w_list)

    plt.figure(figsize=(10,10))
    #plt.hist(rel_w_list, 100, range=(-20,20), color="k", histtype="step", density=True, label='rel w')

    plt.bar(np.linspace(-20,20,1000), w_rel, color="k", label='rel w', alpha=0.1)
    plt.title(f'beta={beta}')
    plt.hist(re_S, 100, range=(-20,20), color="blue", histtype="step", density=True, label='resample', alpha=0.5)

    start_pos = np.array([[s] for s in re_S])
   

    sampler = emcee.EnsembleSampler(K_walkers, dim, log_prob, args=[beta, mu, sigma])
    state = sampler.run_mcmc(start_pos, N_steps)
    S = sampler.flatchain[-alpha:]
    
    
    plt.hist(S, 100, range=(-20,20), color="green", histtype="step", density=True, label='samples', alpha=0.5)
   

    # reference - affine sampler
    p0 = (np.ptp(x)*np.random.rand(K_walkers, dim))-np.max(x)
    sampler = emcee.EnsembleSampler(K_walkers, dim, log_prob, args=[beta, mu, sigma])
    state = sampler.run_mcmc(p0, N_steps)
    samples = sampler.flatchain[burn_in:]
    plt.hist(samples, 100, range=(-20,20), color="k", histtype="step", density=True, label='pdf')
    plt.legend()
    j += 1

    



# bimodal Gaussian distribution

In [ ]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import emcee
import matplotlib.pyplot as plt


### bimodal Gaussian in 1D

def prior(x):
    if np.all(x>=-20) and np.all(x<=20):
        return np.ones(np.size(x))
    else:
        return np.zeros(np.size(x))

    
def likelihood(x, mu, sigma):
    L = 0.5*(1/(sigma[0] * np.sqrt(2 * np.pi))*np.exp(-(x-mu[0])**2 / (2 * sigma[0]**2))) + 0.5*(1/(sigma[1] * np.sqrt(2 * np.pi))*np.exp(-(x-mu[1])**2 / (2 * sigma[1]**2)))
    return L


def log_inf(x):
    return np.where(x != 0, np.log(x), -np.inf) 


def log_prob(x, beta, mu, sigma):
    pr_x = prior(x) 
    like_x = likelihood(x, mu, sigma)
    log_pr = log_inf(pr_x)
    log_like = log_inf(like_x)
    log_post = log_pr + beta*log_like
    return log_post
### 

def OVL(hist1, hist2, edges1, edges2):
    # calculate overlapping coefficient
    assert(len(hist1)==len(hist2))
    assert(np.array_equal(edges1,edges2))
    
    
    bin_width = edges1[1]-edges1[0]
    ovl_coeff = 0.0
    for i in range(len(hist1)):
        ovl_coeff += min(bin_width*hist1[i],bin_width*hist2[i])
    return ovl_coeff
        

### analytical model
def prob(x,beta, mu, sigma):
    pr = 1.0
    L = 0.5*(1/(sigma[0] * np.sqrt(2 * np.pi))*np.exp(-(x-mu[0])**2 / (2 * sigma[0]**2))) + 0.5*(1/(sigma[1] * np.sqrt(2 * np.pi))*np.exp(-(x-mu[1])**2 / (2 * sigma[1]**2)))
    p = pr*(L**beta)
    bin_width = x[1]-x[0]
    
    return (pr*(L**beta)*(bin_width/np.size(x)))


### model configuration
mu = [-10,10]
sigma = [1,2]
x = np.linspace(-20,20,100)
beta = [0, 0.1, 1]
#y_beta = [np.exp(log_prob(x, b, mu, sigma)) for b in beta]  # problem here
y_true = prob(x,1, mu, sigma)
print(sum(y_true))

plt.plot(x,y_true)
plt.show()

### sampling settings
seed = 10
np.random.seed(seed)
dim = 1
max_iter = 100
ESS_min = 500

d_beta = 0.01
N_steps = 100
K_walkers = 10000
NK_total_samples = N_steps*K_walkers
burn_in = 1000

M_sub_samples = NK_total_samples - burn_in #10*K_walkers
assert((M_sub_samples < NK_total_samples) & (M_sub_samples >= 10*K_walkers))


print(f'random seed: {seed}')
print(f'n dim: {dim}')
print(f'max iterations: {max_iter}')
print(f'ESS min: {ESS_min}')
print(f'd Beta: {d_beta}')
print(f'N steps: {N_steps}')
print(f'K walkers: {K_walkers}')
print(f'M sub samples: {M_sub_samples}')
print(f'NK total samples: {NK_total_samples}')
print(f'burn in: {burn_in}')
print(f'#####################')

### step 1: initialization
samples = ((np.ptp(x)*np.random.rand(M_sub_samples))-np.max(x)).flatten()  
beta=0
beta_list = [0.01, 0.1, 0.25, 0.5, 0.75, 1.0, 1.0]

for b in beta_list:
    
    log_p1 = log_prob(samples,beta,mu,sigma)
    beta_new = b
    log_p2 = log_prob(samples, beta_new, mu, sigma)
    log_w = log_p2 - log_p1
    log_w_rel = log_w-np.max(log_w)
    w_rel = np.exp(log_w_rel)
    p_rel = w_rel/np.sum(w_rel)
    print(f'b={b}')
    print(f'sum(w_rel) = {np.sum(w_rel)}')
    print(f'mean(w_rel) = {np.mean(w_rel)}')

    resamples = np.random.choice(samples,size=K_walkers,p=p_rel)
    resamples_hist, resamples_edges = np.histogram(resamples, 100, range=(-20,20), density=True)

    p0 = np.array([[s] for s in resamples])

    sampler = emcee.EnsembleSampler(K_walkers, dim, log_prob, args=[beta_new, mu, sigma])
    state = sampler.run_mcmc(p0, N_steps)
    samples = (sampler.flatchain[burn_in:]).flatten()
    samples_hist, samples_edges = np.histogram(samples, 100, range=(-20,20), density=True)
    #assert(np.size(samples)==M_sub_samples)

    # reference - affine sampler
    p0_ref = (np.ptp(x)*np.random.rand(K_walkers, dim))-np.max(x)
    sampler_ref = emcee.EnsembleSampler(K_walkers, dim, log_prob, args=[beta_new, mu, sigma])
    state_ref = sampler_ref.run_mcmc(p0_ref, N_steps)
    samples_ref = sampler_ref.flatchain[burn_in:]
    samples_ref_hist, samples_ref_edges = np.histogram(samples_ref, 100, range=(-20,20), density=True)

    
    
    print(f'old beta = {beta}')
    print(f'new beta = {beta_new}')
    print(f'number of resamples: {np.size(resamples)}')
    print(f'number of samples: {np.size(samples)}')
    print(f'number of samples (ref): {np.size(samples_ref)}')
    print(f'overlapping coefficient (resamples vs samples): {OVL(resamples_hist,samples_hist,resamples_edges,samples_edges)}' )
    print(f'overlapping coefficient (resamples vs samples ref): {OVL(resamples_hist,samples_ref_hist,resamples_edges,samples_ref_edges)}' )
    print(f'overlapping coefficient (samples vs samples ref): {OVL(samples_hist,samples_ref_hist,samples_edges,samples_ref_edges)}' )
    print(f'\n')
    
    plt.figure(figsize=(10,10))
    plt.title(f'beta={beta_new}')
    #print(np.shape(x))
    #print(np.shape(p_rel))
    #plt.bar(x, p_rel, color='red', label='rel prob', alpha=0.55)
    plt.hist(resamples, 100, range=(-20,20), color="green", histtype="step", density=True, label='resamples', alpha=0.75)
    plt.hist(samples, 100, range=(-20,20), color="blue", histtype="step", density=True, label='sub-samples', alpha=0.75)
    plt.hist(samples_ref, 100, range=(-20,20), color="black", histtype="step", density=True, label='ref samples', alpha=0.75)
    
    
    plt.legend()
    
    beta=beta_new

# 2D transporter model

In [ ]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import emcee
import matplotlib.pyplot as plt
import tellurium as te
import time


### 2d transporter

def make_2d_prob_grid(n_grid, a_range, b_range, beta, K_true, y_obs, m):
    
    log_like_ref = calc_grid_point([K_true[0],K_true[1]],y_obs,m)

    n_grid = 100

    a_vals = np.linspace(a_range[0],a_range[1],n_grid)
    b_vals = np.linspace(b_range[0],b_range[1],n_grid)


    a_tmp, b_tmp = np.meshgrid(a_vals,b_vals)
    Z = np.zeros((n_grid,n_grid))
    log_like_max = np.NINF

    for i in range(n_grid):
        for j in range(n_grid):
            K = [a_tmp[i,j],b_tmp[i,j]]
            Z[i,j] = beta*calc_grid_point(K,y_obs,m)  # log probability
            #print(K[0],K[1],Z[i,j])
            if Z[i,j] > log_like_max:
                log_like_max = Z[i,j]

    print(f'log-likelihood: ref={log_like_ref}, grid max={log_like_max}')
    fig = plt.figure(figsize=(10,10))                    
    c = plt.pcolor(a_tmp, b_tmp, Z, shading='auto', vmin=log_like_max-5, vmax=log_like_max)
    #fig.colorbar(c)
    plt.title(f'log likelihood reference - beta={beta}')
    plt.xlabel('p_1')
    plt.ylabel('p_2')
    plt.show()


# Normal log-likelihood calculation
def calc_norm_log_likelihood(mu,sigma,X):
    # Normal log-likelihood function: -[(n/2)ln(2pp*sigma^2)]-[sum((X-mu)^2)/(2*sigma^2)]
    # ref: https://www.statlect.com/fundamentals-of-statistics/normal-distribution-maximum-likelihood 
    n = len(X)
    f1 = -1*(n/2)*np.log(2*np.pi*sigma**2)
    f2_a = -1/(2*sigma**2)
    f2_b = 0 
    for i in range(n):
        f2_b += (X[i]-mu[i])**2
    f2 = f2_a*f2_b
    log_likelihood = f1+f2
    return log_likelihood


def calc_grid_point(K,y_obs,m):
    m.resetToOrigin()
    m.H_out = 5e-8
    m.integrator.absolute_tolerance = 1e-18
    m.integrator.relative_tolerance = 1e-12
    m.k1_f = 10**K[0]
    m.k1_r = 10**K[1]
    m.k6_r = (m.k1_f*m.k2_f*m.k3_f*m.k4_f*m.k5_f*m.k6_f)/(m.k1_r*m.k2_r*m.k3_r*m.k4_r*m.k5_r)
 
    D_tmp = m.simulate(0, 10, 250, selections=['time', 'rxn4'])
    y_tmp = D_tmp['rxn4']
    sigma = 1e-13
    log_like_tmp = calc_norm_log_likelihood(y_tmp,sigma,y_obs)
    return log_like_tmp


def log_prior(theta):
    '''log of uniform prior distribution'''
    p0 = theta[0]
    p1 = theta[1]

    # if prior is between boundary --> log(prior) = 0 (uninformitive prior)
    if (6<p0<12) and (-1<p1<5):
        return 0  
    else:
        return -np.inf


def log_prob(theta, y_obs, extra_parameters):
    '''log of estimated posterior probability'''
    m = extra_parameters[0]
    beta = extra_parameters[1]
    log_pr = log_prior(theta)
    if not np.isfinite(log_pr):
        return -np.inf  # ~zero probability
    log_prob = log_pr + beta*calc_grid_point(theta, y_obs, m)  # log posterior ~ log likelihood + log prior
    return log_prob


def set_p0():
    '''set initial walker positions'''
    log_k1_f = np.random.uniform(6, 12) # log10 rate constant (ref=1e10)
    log_k1_r = np.random.uniform(-1, 5)  # log10 rate constant (ref=1e3)    
    p0_list_tmp = [              
                log_k1_f ,
                log_k1_r ,
    ]
    return p0_list_tmp


### model configuration
print('model configuration...')
antimony_string_SS = f"""
            // Created by libAntimony v2.12.0
            model transporter_full()

            // Compartments and Species:
            compartment vol;
            species OF in vol, OF_Hb in vol;
            species IF_Hb in vol, IF_Hb_Sb in vol;
            species IF_Sb in vol, OF_Sb in vol;
            species H_in in vol, S_in in vol;
            species $H_out in vol, $S_out in vol;

            // Reactions:
            rxn1: OF + $H_out -> OF_Hb; vol*(k1_f*OF*H_out - k1_r*OF_Hb);
            rxn2: OF_Hb -> IF_Hb; vol*(k2_f*OF_Hb - k2_r*IF_Hb);
            rxn3: IF_Hb + S_in -> IF_Hb_Sb; vol*(k3_f*IF_Hb*S_in - k3_r*IF_Hb_Sb);
            rxn4: IF_Hb_Sb -> IF_Sb + H_in; vol*(k4_f*IF_Hb_Sb - k4_r*IF_Sb*H_in);
            rxn5: IF_Sb -> OF_Sb; vol*(k5_f*IF_Sb - k5_r*OF_Sb);
            rxn6: OF_Sb -> OF + $S_out; vol*(k6_f*OF_Sb - k6_r*OF*S_out);
            

            // Events:
            E1: at (time >= 5): H_out = 1e-7, S_out = 1e-3;
            

            // Species initializations:
            H_out = 5e-8;
            H_out has substance_per_volume;

            H_in = 9.999811082242941e-08;
            H_in has substance_per_volume;

            S_out = 0.001;
            S_out has substance_per_volume;

            S_in = 0.0009999811143288836;
            S_in has substance_per_volume;

            OF = 4.7218452046117796e-09;
            OF has substance_per_volume;

            OF_Hb = 4.7218452046117796e-09;
            OF_Hb has substance_per_volume;

            IF_Hb = 4.7218452046117796e-09;
            IF_Hb has substance_per_volume;
            
            IF_Hb_Sb = 4.721756029392908e-08;
            IF_Hb_Sb has substance_per_volume;
            
            IF_Sb = 4.721845204611779e-08;
            IF_Sb has substance_per_volume;

            OF_Sb = 4.721845204611775e-08;
            OF_Sb has substance_per_volume;


            // Compartment initializations:
            vol = 0.0001;
            vol has volume;

            // Rate constant initializations:
            k1_f = 1e10;
            k1_r = 1e3;
            k2_f = 1e2;
            k2_r = 1e2;
            k3_f = 1e7;
            k3_r = 1e3;
            k4_f = 1e3;
            k4_r = 1e10;
            k5_f = 1e2;
            k5_r = 1e2;
            k6_f = 1e3;
            k6_r = 1e7;


            // Other declarations:
            const vol;
            const k1_f, k1_r, k2_f, k2_r, k3_f, k3_r;
            const k4_f, k4_r, k5_f, k5_r, k6_f, k6_r;
    

            // Unit definitions:
            unit substance_per_volume = mole / litre;
            unit volume = litre;
            unit length = metre;
            unit area = metre^2;
            unit time_unit = second;
            unit substance = mole;
            unit extent = mole;

            // Display Names:
            time_unit is "time";
            end
""" 
           
m = te.loada(antimony_string_SS)
m.integrator.absolute_tolerance = 1e-18
m.integrator.relative_tolerance = 1e-12
m.H_out = 5e-8
D1 = m.simulate(0, 10, 250, selections=['time', 'rxn4'])
y_true = D1['rxn4']

noise_stdev_true = 1e-13
y_obs = np.genfromtxt("data_grid_test3_1exp.csv")

plt.figure(figsize=(10,10))
plt.plot(y_obs, 'o', alpha=0.5)
plt.plot(y_true)
plt.ylim(-2.5e-11, 2.5e-11)
plt.ylabel('y')
plt.xlabel('t')



# beta_test = [1e-6, 1e-4, 0.001, 0.01, 0.1, 0.25, 0.5, 0.75, 1.0]

# for b in beta_test:
#     make_2d_prob_grid(n_grid=100, a_range=[6,12], b_range=[-1,5], beta=b, K_true=[10,3], y_obs=y_obs, m=m )

print(f'log_prob_ref: {log_probability([10,3], y_obs, [m,1])}')


#### sampling settings
t0 = time.time()
print(f't0 timestamp: {t0}')
print('sampling configuration...')
seed = 10
np.random.seed(seed)
dim = 2
max_iter = 1e5
ESS_min = 500
fractional_weight_target = 0.1

d_beta = 1e-5
N_steps = 100
K_walkers = 100
NK_total_samples = N_steps*K_walkers
burn_in = 1000

M_sub_samples = NK_total_samples - burn_in #10*K_walkers
assert((M_sub_samples < NK_total_samples) & (M_sub_samples >= 10*K_walkers))

print(f'random seed: {seed}')
print(f'n dim: {dim}')
print(f'max iterations: {max_iter}')
print(f'ESS min: {ESS_min}')
print(f'd Beta: {d_beta}')
print(f'fractional weight target: {fractional_weight_target}')
print(f'N steps: {N_steps}')
print(f'K walkers: {K_walkers}')
print(f'M sub samples: {M_sub_samples}')
print(f'NK total samples: {NK_total_samples}')
print(f'burn in: {burn_in}')
print(f'#####################')


### step 1: initialization
print('initializing...')
beta=0
#beta_list = [1e-6, 1e-4, 0.001, 0.01, 0.1, 0.25, 0.5, 0.75, 1.0]
#beta_list = [1e-6, 1e-4, 0.001, 0.01, 0.1]

pos_list=[]
for i in range(K_walkers):
    p0_list_tmp = set_p0()
    pos_list.append(p0_list_tmp)
samples = np.asarray(pos_list)
assert(np.shape(samples)==(K_walkers, dim))
print(f'initial walker shape: {np.shape(samples)}')
print(f'p0: {samples[0:2]}')

iter_i = 0
while (beta <=1) and (iter_i < max_iter):
    print(f'i={iter_i}')
    
    log_like = np.array([log_prob(theta_i,y_obs,[m,1]) for theta_i in samples])
    mean_w_rel = 0
    iter_j = 0
    beta_new = beta
    tmp_beta_list = []
    
    print('finding new beta...')
    beta_new = beta_new+d_beta
    log_w = (beta_new-beta)*log_like
    log_w_rel = log_w-np.max(log_w)
    w_rel = np.exp(log_w_rel)
    p_rel = w_rel/np.sum(w_rel)
    mean_w_rel = np.mean(w_rel)
#     print(f'  beta_new={beta_new}')
#     print(f'  sum(w_rel) = {np.sum(w_rel)}')
#     print(f'  mean(w_rel) = {np.mean(w_rel)}')

#     print(f'  p_rel shape: {np.shape(p_rel)}')
#     print(f'  p_rel: {p_rel[0:2]}')
#     print(f'  max p_rel: {max(p_rel)}')
#     print(f'  argmax p_rel: {np.argmax(p_rel)}')
    assert not mean_w_rel < fractional_weight_target, f"mean fractional weight {mean_w_rel} < target {fractional_weight_target} with smallest step size {d_beta}. use smaller step size"
    tmp_beta_list.append(beta_new)
    
    while (mean_w_rel>=fractional_weight_target) and (iter_j < max_iter) and beta_new <1:

        beta_new = beta_new+d_beta
        log_w = (beta_new-beta)*log_like
        log_w_rel = log_w-np.max(log_w)
        w_rel = np.exp(log_w_rel)
        p_rel = w_rel/np.sum(w_rel)
        mean_w_rel = np.mean(w_rel)
#         print(f'  j = {iter_j}')
#         print(f'  beta_new={beta_new}')
#         print(f'  sum(w_rel) = {np.sum(w_rel)}')
#         print(f'  mean(w_rel) = {np.mean(w_rel)}')

#         print(f'  p_rel shape: {np.shape(p_rel)}')
#         print(f'  p_rel: {p_rel[0:2]}')
#         print(f'  max p_rel: {max(p_rel)}')
#         print(f'  argmax p_rel: {np.argmax(p_rel)}')
        iter_j = iter_j + 1
        #print(f'  ')
        tmp_beta_list.append(beta_new)
    print(f'new beta = {beta_new} after {iter_j} iterations with mean(w_rel) = {mean_w_rel}')

    resamples_index = np.random.choice([ i for i in range(len(samples))],size=K_walkers,p=p_rel)
    resamples = samples[resamples_index]
    resample_x = [i[0] for i in resamples ]
    resample_y = [i[1] for i in resamples ]
    if beta_new > 0.1:
        plt.figure(figsize=(10,10))
        plt.title(f'log likelihood resample - beta={beta_new}')
        resamples_hist, resamples_x_edges, resamples_y_edges, resamples_image = plt.hist2d(resample_x, resample_y, 100, density=True, range=([[6, 12], [-1, 5]]))
        plt.xlabel('p2')
        plt.ylabel('p1')
    #plt.colorbar()
    #print(f'resample shape: {np.shape(resamples)}')

    p0 = np.array([s for s in resamples])
    #print(f'resampled p0 shape: {np.shape(p0)}')

    print(f'running Affine sampler...')
    sampler = emcee.EnsembleSampler(K_walkers, dim, log_prob, args=[y_obs, [m,beta_new]])
    state = sampler.run_mcmc(p0, N_steps)
    samples = sampler.flatchain[burn_in:]
    print(np.shape(samples))
    samples_x = [i[0] for i in samples]
    samples_y = [i[1] for i in samples]
    if beta_new > 0.1:
        plt.figure(figsize=(10,10))
        plt.title(f'log likelihood sampled - beta={beta_new}')
        samples_hist, samples_x_edges, samples_y_edges, samples_image = plt.hist2d(samples_x, samples_y, 100, density=True, range=([[6, 12], [-1, 5]]))
        plt.xlabel('p2')
        plt.ylabel('p1')
    #plt.colorbar()
    #print(f'sample shape: {np.shape(samples)}')
    if beta_new > 0.1:
        print(f'calculating reference...')
        make_2d_prob_grid(n_grid=100, a_range=[6,12], b_range=[-1,5], beta=beta_new, K_true=[10,3], y_obs=y_obs, m=m )
    iter_i = iter_i +1
    beta=beta_new


tf = time.time()
print(f'tf timestamp: {tf}')
print(f'wall clock: {tf-t0} s')
print(f'{(len(beta_list)*NK_total_samples)/(tf-t0)} samples/sec' )

# 12D transporter model

In [ ]:
import numpy as np
import scipy.stats as stats, scipy as sp
import seaborn as sns
import emcee
import matplotlib.pyplot as plt
import tellurium as te
import time
import corner as corner


### 12d transporter

# Normal log-likelihood calculation
def calc_norm_log_likelihood(mu,sigma,X):
    # Normal log-likelihood function: -[(n/2)ln(2pp*sigma^2)]-[sum((X-mu)^2)/(2*sigma^2)]
    # ref: https://www.statlect.com/fundamentals-of-statistics/normal-distribution-maximum-likelihood 
    n = len(X)
    f1 = -1*(n/2)*np.log(2*np.pi*sigma**2)
    f2_a = -1/(2*sigma**2)
    f2_b = 0 
    for i in range(n):
        f2_b += (X[i]-mu[i])**2
    f2 = f2_a*f2_b
    log_likelihood = f1+f2
    return log_likelihood


def calc_grid_point(K,y_obs,m):
    m.resetToOrigin()
    m.H_out = 5e-8
    m.integrator.absolute_tolerance = 1e-18
    m.integrator.relative_tolerance = 1e-12
    m.k1_f = 10**K[0]
    m.k1_r = 10**K[1]
    m.k2_f = 10**K[2]
    m.k2_r = 10**K[3]
    m.k3_f = 10**K[4]
    m.k3_r = 10**K[5]
    m.k4_f = 10**K[6]
    m.k4_r = 10**K[7]
    m.k5_f = 10**K[8]
    m.k5_r = 10**K[9]
    m.k6_f = 10**K[10]
    m.k6_r = (m.k1_f*m.k2_f*m.k3_f*m.k4_f*m.k5_f*m.k6_f)/(m.k1_r*m.k2_r*m.k3_r*m.k4_r*m.k5_r)
    try:
        D_tmp = m.simulate(0, 10, 250, selections=['time', 'rxn4'])
        y_tmp = D_tmp['rxn4']
        sigma = 10**K[11]
        log_like_tmp = calc_norm_log_likelihood(y_tmp,sigma,y_obs)
    except:
        y_tmp = np.zeros(250)
        sigma = 10**K[11]
        log_like_tmp = calc_norm_log_likelihood(y_tmp,sigma,y_obs)
    return log_like_tmp


def log_prior(theta):
    '''log of uniform prior distribution'''
    
    p1 = theta[0]
    p2 = theta[1]
    p3 = theta[2]
    p4 = theta[3]
    p5 = theta[4]
    p6 = theta[5]
    p7 = theta[6]
    p8 = theta[7]
    p9 = theta[8]
    p10 = theta[9]
    p11 = theta[10]
    p_sigma = theta[11]

    # if prior is between boundary --> log(prior) = 0 (uninformitive prior)
    if (np.log10(5e-14)<p_sigma<np.log10(5e-13)) and (6<p1<12) and (-1<p2<5) and (-2<p3<4) and (-2<p4<4) and \
        (3<p5<9) and (-1<p6<5) and (-1<p7<5) and (6<p8<12) and (-2<p9<4) and (-2<p10<4) and (-1<p11<5):
        return 0  
    else:
        return -np.inf
    

def log_prob(theta, y_obs, extra_parameters):
    '''log of estimated posterior probability'''
    m = extra_parameters[0]
    beta = extra_parameters[1]
    log_pr = log_prior(theta)
    log_l = calc_grid_point(theta, y_obs, m)
    if not np.isfinite(log_pr):
        return -np.inf  # ~zero probability
    if not np.isfinite(log_l):
        return -np.inf  # ~zero probability
    else:
        log_prob = log_pr + beta*log_l  # log posterior ~ log likelihood + log prior
        return log_prob


def set_p0():
    '''set initial walker positions'''
    
    log_k1_f = np.random.uniform(6, 12) # log10 rate constant (ref=1e10)
    log_k1_r = np.random.uniform(-1,5)  # log10 rate constant (ref=1e3) 
    log_k2_f = np.random.uniform(-2,4)  # log10 rate constant (ref=1e2)
    log_k2_r = np.random.uniform(-2,4)  # log10 rate constant (ref=1e2)

    log_k3_f = np.random.uniform(3,9)  # log10 rate constant (ref=1e7) 
    log_k3_r = np.random.uniform(-1,5)  # log10 rate constant (ref=1e3) 
    log_k4_f = np.random.uniform(-1,5)  # log10 rate constant (ref=1e3) 
    log_k4_r = np.random.uniform(6, 12)  # log10 rate constant (ref=1e10)
    log_k5_f = np.random.uniform(-2,4)  # log10 rate constant (ref=1e2)
    log_k5_r = np.random.uniform(-2,4)   # log10 rate constant (ref=1e2)
    log_k6_f = np.random.uniform(-1,5)  # log10  rate constant (ref=1e3)
    log_noise_sigma = np.random.uniform(np.log10(5e-14), np.log10(5e-13))
    
    p0_list_tmp = [        
                log_k1_f ,
                log_k1_r ,
                log_k2_f ,
                log_k2_r ,
                log_k3_f , 
                log_k3_r ,
                log_k4_f ,
                log_k4_r ,
                log_k5_f ,
                log_k5_r ,
                log_k6_f ,
                log_noise_sigma ,
    ]
    return p0_list_tmp


def plot_samples(samples, p_info, beta):
    
    ### 1D posterior plot
    samples_T = np.transpose(samples)
    fig, ax = plt.subplots(3,4, figsize=(20,15))
    axes = ax.flatten()

    for i, ax_i in enumerate(axes):
        p_tmp = p_info[i]
        ax_i.set_title(f"{p_tmp[0]}")
        ax_i.axvline(p_tmp[3],ls='--', color='black', alpha=0.5)
        ax_i.set_xlim(p_tmp[1],p_tmp[2])


        ax_i.hist(samples_T[i], 100, histtype="step", density=True, range=(p_tmp[1],p_tmp[2]), label=f'{p_tmp[0]}')
        ax_i.legend()

    plt.suptitle(f'1D distributions - beta={beta}')
    plt.tight_layout()
    
    #plt.savefig(f'12D_transporter_AIES_PT_1D_dist_s{seed}_random.png') 
    

def calculate_weights(log_like, beta_old, beta_new):
    log_w = (beta_new-beta)*log_like
    log_w_rel = log_w-np.max(log_w)
    w_rel = np.exp(log_w_rel)
    return w_rel
    

def calculate_next_beta(log_like, beta_old, threshold):
    def f(x):
        avg_w_rel =  np.mean(calculate_weights(log_like, beta_old, x))    
        return avg_w_rel-threshold
    
    beta_new = sp.optimize.root(f, beta_old).x[0]
    assert(beta_old <= beta_new)
    if beta_new >= 1.0:
        beta_new = 1.0
    else:
        assert(np.isclose(np.mean(calculate_weights(log_like, beta_old, beta_new)),threshold))
    return beta_new


def calculate_p_rel(log_like, beta_old, beta_new):
    w_rel = calculate_weights(log_like, beta, beta_new)
    mean_w_rel = np.mean(w_rel)
    p_rel = w_rel/np.sum(w_rel)
    assert(np.isclose(np.sum(p_rel),1.0))
    return p_rel
    
    
### model configuration
print('model configuration...')
antimony_string_SS = f"""
            // Created by libAntimony v2.12.0
            model transporter_full()

            // Compartments and Species:
            compartment vol;
            species OF in vol, OF_Hb in vol;
            species IF_Hb in vol, IF_Hb_Sb in vol;
            species IF_Sb in vol, OF_Sb in vol;
            species H_in in vol, S_in in vol;
            species $H_out in vol, $S_out in vol;

            // Reactions:
            rxn1: OF + $H_out -> OF_Hb; vol*(k1_f*OF*H_out - k1_r*OF_Hb);
            rxn2: OF_Hb -> IF_Hb; vol*(k2_f*OF_Hb - k2_r*IF_Hb);
            rxn3: IF_Hb + S_in -> IF_Hb_Sb; vol*(k3_f*IF_Hb*S_in - k3_r*IF_Hb_Sb);
            rxn4: IF_Hb_Sb -> IF_Sb + H_in; vol*(k4_f*IF_Hb_Sb - k4_r*IF_Sb*H_in);
            rxn5: IF_Sb -> OF_Sb; vol*(k5_f*IF_Sb - k5_r*OF_Sb);
            rxn6: OF_Sb -> OF + $S_out; vol*(k6_f*OF_Sb - k6_r*OF*S_out);
            

            // Events:
            E1: at (time >= 5): H_out = 1e-7, S_out = 1e-3;
            

            // Species initializations:
            H_out = 5e-8;
            H_out has substance_per_volume;

            H_in = 9.999811082242941e-08;
            H_in has substance_per_volume;

            S_out = 0.001;
            S_out has substance_per_volume;

            S_in = 0.0009999811143288836;
            S_in has substance_per_volume;

            OF = 4.7218452046117796e-09;
            OF has substance_per_volume;

            OF_Hb = 4.7218452046117796e-09;
            OF_Hb has substance_per_volume;

            IF_Hb = 4.7218452046117796e-09;
            IF_Hb has substance_per_volume;
            
            IF_Hb_Sb = 4.721756029392908e-08;
            IF_Hb_Sb has substance_per_volume;
            
            IF_Sb = 4.721845204611779e-08;
            IF_Sb has substance_per_volume;

            OF_Sb = 4.721845204611775e-08;
            OF_Sb has substance_per_volume;


            // Compartment initializations:
            vol = 0.0001;
            vol has volume;

            // Rate constant initializations:
            k1_f = 1e10;
            k1_r = 1e3;
            k2_f = 1e2;
            k2_r = 1e2;
            k3_f = 1e7;
            k3_r = 1e3;
            k4_f = 1e3;
            k4_r = 1e10;
            k5_f = 1e2;
            k5_r = 1e2;
            k6_f = 1e3;
            k6_r = 1e7;


            // Other declarations:
            const vol;
            const k1_f, k1_r, k2_f, k2_r, k3_f, k3_r;
            const k4_f, k4_r, k5_f, k5_r, k6_f, k6_r;
    

            // Unit definitions:
            unit substance_per_volume = mole / litre;
            unit volume = litre;
            unit length = metre;
            unit area = metre^2;
            unit time_unit = second;
            unit substance = mole;
            unit extent = mole;

            // Display Names:
            time_unit is "time";
            end
""" 
seed = 10
np.random.seed(seed)

m = te.loada(antimony_string_SS)
m.integrator.absolute_tolerance = 1e-18
m.integrator.relative_tolerance = 1e-12
m.H_out = 5e-8
D1 = m.simulate(0, 10, 250, selections=['time', 'rxn4'])
y_true = D1['rxn4']

noise_stdev_true = 1e-13
y_obs = np.genfromtxt("data_grid_test3_1exp.csv")

plt.figure(figsize=(10,10))
plt.plot(y_obs, 'o', alpha=0.5)
plt.plot(y_true)
plt.ylim(-2.5e-11, 2.5e-11)
plt.ylabel('y')
plt.xlabel('t')

p_info = [   
    ["log_k1_f",6,12,10],
    ["log_k1_r",-1,5,3],
    ["log_k2_f",-2,4,2],
    ["log_k2_r",-2,4,2],
    ["log_k3_f",3,9,7],
    ["log_k3_r",-1,5,3],
    ["log_k4_f",-1,5,3],
    ["log_k4_r",6,12,10],
    ["log_k5_f",-2,4,2],
    ["log_k5_r",-2,4,2],
    ["log_k6_f",-1,5,3],
    ["log_sigma",np.log10(5e-14), np.log10(5e-13), -13],
]
param_ref = [p_i[3] for p_i in p_info]
print(param_ref)

# beta_test = [1e-6, 1e-4, 0.001, 0.01, 0.1, 0.25, 0.5, 0.75, 1.0]

# for b in beta_test:

#     make_2d_prob_grid(n_grid=100, a_range=[6,12], b_range=[-1,5], beta=b, K_true=[10,3], y_obs=y_obs, m=m )

print(f'log_prob_ref: {log_prob(param_ref, y_obs, [m,1])}')


#### sampling settings
t0 = time.time()
print(f't0 timestamp: {t0}')
print('sampling configuration...')
seed = 10
np.random.seed(seed)
dim = 12
max_iter = 1e5
fractional_weight_target = 0.999

d_beta = 1e-5
N_steps = 100
K_walkers = 100
NK_total_samples = N_steps*K_walkers
burn_in = 1000

M_sub_samples = NK_total_samples - burn_in #10*K_walkers
assert((M_sub_samples < NK_total_samples) & (M_sub_samples >= 10*K_walkers))

print(f'random seed: {seed}')
print(f'n dim: {dim}')
print(f'max iterations: {max_iter}')
print(f'd Beta: {d_beta}')
print(f'fractional weight target: {fractional_weight_target}')
print(f'N steps: {N_steps}')
print(f'K walkers: {K_walkers}')
print(f'M sub samples: {M_sub_samples}')
print(f'NK total samples: {NK_total_samples}')
print(f'burn in: {burn_in}')
print(f'#####################')


### step 1: initialization
print('initializing...')
beta=0.0

pos_list=[]
for i in range(K_walkers):
    p0_list_tmp = set_p0()
    pos_list.append(p0_list_tmp)
samples = np.asarray(pos_list)
assert(np.shape(samples)==(K_walkers, dim))
print(f'initial walker shape: {np.shape(samples)}')
print(f'p0: {samples[0:2]}')

print('sampling...')
iter_i = 0
while (beta <1) and (iter_i < max_iter):
   

    log_like = np.array([log_prob(theta_i,y_obs,[m,1]) for theta_i in samples])
    beta_new = calculate_next_beta(log_like, beta, fractional_weight_target)
    p_rel = calculate_p_rel(log_like, beta, beta_new)
    

    resamples_index = np.random.choice([ i for i in range(len(samples))],size=K_walkers,p=p_rel)
    resamples = samples[resamples_index]

    p0 = np.array([s for s in resamples])
    assert(np.shape(p0)==(K_walkers, dim))

    
    sampler = emcee.EnsembleSampler(K_walkers, dim, log_prob, args=[y_obs, [m,beta_new]])
    state = sampler.run_mcmc(p0, N_steps)
    samples = sampler.flatchain[burn_in:]
    
    if beta_new > 0.9:
        plot_samples(samples,p_info, beta_new)
    
    print(f'i={iter_i}, new beta={beta_new:.2g}, old beta={beta:.2g}, delta beta={(beta_new-beta):.2g}')
    
    iter_i = iter_i +1
    beta=beta_new
    

tf = time.time()
print(f'{iter_i} beta steps')
print(f'tf timestamp: {tf}')
print(f'wall clock: {tf-t0} s')
print(f'{(iter_i*NK_total_samples)/(tf-t0)} samples/sec' )

In [ ]:
import pandas as pd
df = pd.DataFrame(samples, columns=[i[0] for i in p_info])
df.to_csv(f'ais_affine_{K_walkers}w_{N_steps}s_{fractional_weight_target}t_{seed}r.csv')
